In [1]:
from pyspark.sql import functions as F 
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel

In [2]:
list=[['cm',30.0,50.0],
     ['cm',40.0,70.0],
     ['cm',20.0,85.0],
     ['cm',20.0,85.0],
     ['cm',30.0,76.0],
     ['cm',43.0,65.0],
     ['Meter',0.2,.65],
     ['Meter',0.4,.68],
     ['Meter',0.5,.8]]

df=spark.createDataFrame(list,['unit','ref_low','ref_high'])

df.show()

+-----+-------+--------+
 unit|ref_low|ref_high|
+-----+-------+--------+
 cm| 30.0| 50.0|
 cm| 40.0| 70.0|
 cm| 20.0| 85.0|
 cm| 20.0| 85.0|
 cm| 30.0| 76.0|
 cm| 43.0| 65.0|
Meter| 0.2| 0.65|
Meter| 0.4| 0.68|
Meter| 0.5| 0.8|
+-----+-------+--------+

In [3]:
#I wanted to convert some of the cm values (ref_low = 20 and ref_low <40, & ref_high > 70 and ref_high <90,) to meter using the formula (cm/100). I tried to use Pyspark UDF

from pyspark.sql import functions as F
df.withColumn("ref_low", F.when((F.col("unit")=='cm')&((F.col("ref_low")<40)&\
                                 (F.col("ref_low")==20)), F.col("ref_low")/100)\
             .otherwise(F.col("ref_low")))\
  .withColumn("ref_high", F.when((F.col("unit")=='cm')&((F.col("ref_high")<90)&\
                                  (F.col("ref_high")>70)),F.col("ref_high")/100)\
              .otherwise(F.col("ref_high"))).show()

+-----+-------+--------+
 unit|ref_low|ref_high|
+-----+-------+--------+
 cm| 30.0| 50.0|
 cm| 40.0| 70.0|
 cm| 0.2| 0.85|
 cm| 0.2| 0.85|
 cm| 30.0| 0.76|
 cm| 43.0| 65.0|
Meter| 0.2| 0.65|
Meter| 0.4| 0.68|
Meter| 0.5| 0.8|
+-----+-------+--------+

In [4]:
list1=[[1,-2],
      [1,-2],
      [1,2],
      [1,3],
      [1,4]]

list2=[[2,3],
       [3,4]]

dataframe=spark.createDataFrame(list1,['col1','col2'])

df=spark.createDataFrame(list2,['col1','col2'])





In [5]:
dataframe.join(df.select("col2"),["col2"],how='left_anti').show()

+----+----+
col2|col1|
+----+----+
 -2| 1|
 -2| 1|
 2| 1|
+----+----+

In [6]:
dataframe.filter(~F.col("col2").isin([x[0] for x in df.select("col2").collect()])).show()

+----+----+
col1|col2|
+----+----+
 1| -2|
 1| -2|
 1| 2|
+----+----+

In [7]:
df.filter(~F.col("col2").isin([x[0] for x in df1.select("col2").collect()])).show()


+----+----+
col1|col2|
+----+----+
 1| 2|
 1| 2|
 1| 2|
+----+----+

In [8]:
df.except(df1.select("col2")).show()

File "<command-4348710841451748>" , line 1 
 df.except(df1.select("col2")).show() 
 ^
 SyntaxError : invalid syntax

In [9]:
df.filter(~F.col("col2").isin(b)).show()

+----+----+
col1|col2|
+----+----+
 1| 2|
 1| 2|
 1| 2|
+----+----+

In [10]:
df = spark.createDataFrame(
    [(1, ["foo", "bar"], {"x": 1.0}), (2, [], {}), (3, None, None)],
    ("id", "an_array", "a_map")
)
df.show()
df.select("id", "an_array", F.explode_outer("a_map")).show()
df.select("id", "an_array", F.explode("a_map")).show()

+---+----------+----------+
 id| an_array| a_map|
+---+----------+----------+
 1|[foo, bar]|[x -> 1.0]|
 2| []| []|
 3| null| null|
+---+----------+----------+

+---+----------+----+-----+
 id| an_array| key|value|
+---+----------+----+-----+
 1|[foo, bar]| x| 1.0|
 2| []|null| null|
 3| null|null| null|
+---+----------+----+-----+

+---+----------+---+-----+
 id| an_array|key|value|
+---+----------+---+-----+
 1|[foo, bar]| x| 1.0|
+---+----------+---+-----+

In [11]:
list=[['A','2020-04-08T01:53:54.932000','Org1','wifi',60,372717],
                      ['A','2020-04-08T02:40:38.661000','Org1','wifi',194,819040],
                       ['A','2020-04-08T21:45:10.207000','Org1','wifi',8885,3449150],
                        ['A','2020-04-09T00:15:28.838000','Org1','wifi',14770,3572589],
                         ['A','2020-04-09T04:27:33.424000','Org1','remote',0,0],
                          ['A','2020-04-09T04:29:25.189000','Org1','wifi',60,7495],
                           ['A','2020-04-09T04:44:21.397000','Org1','remote',60,553356],
                            ['A','2020-04-09T04:50:40.406000','Org1','wifi',60,662467],
                             ['A','2020-04-10T00:00:50.636000','Org1','remote',0,72],
                              ['A','2020-04-10T04:20:28.831000','Org1','remote',6,497],
                               ['A','2020-04-10T04:31:35.336000','Org1','remote',0,22],
                                ['B','2020-04-08T21:56:58.502000','Org2','remote',0,0],
                                 ['B','2020-04-08T22:01:19.534000','Org2','wifi',0,0],
                                  ['B','2020-04-08T22:10:15.891000','Org2','wifi',60,187891],
                                   ['B','2020-04-08T22:16:41.499000','Org2','wifi',1620,207674],
                                    ['B','2020-04-09T01:55:02.772000','Org2','wifi',360,426232],
                                     ['B','2020-04-09T02:03:32.735000','Org2','wifi',60,374827],
                                      ['B','2020-04-09T02:06:16.509000','Org2','wifi',60,386518],
                                       ['B','2020-04-09T02:13:33.497000','Org2','remote',60,373609],
                                        ['B','2020-04-09T02:17:19.176000','Org2','wifi',133,400417],
                                         ['B','2020-04-10T23:10:15.654000','Org2','remote',0,212],
                                          ['B','2020-04-10T23:10:41.749000','Org2','remote',1,285]]

df=spark.createDataFrame(list,["deviceId","time-started","OrgId","type","duration","packet"])


df.show(truncate=False)


+--------+--------------------------+-----+------+--------+-------+
deviceId|time-started |OrgId|type |duration|packet |
+--------+--------------------------+-----+------+--------+-------+
A |2020-04-08T01:53:54.932000|Org1 |wifi |60 |372717 |
A |2020-04-08T02:40:38.661000|Org1 |wifi |194 |819040 |
A |2020-04-08T21:45:10.207000|Org1 |wifi |8885 |3449150|
A |2020-04-09T00:15:28.838000|Org1 |wifi |14770 |3572589|
A |2020-04-09T04:27:33.424000|Org1 |remote|0 |0 |
A |2020-04-09T04:29:25.189000|Org1 |wifi |60 |7495 |
A |2020-04-09T04:44:21.397000|Org1 |remote|60 |553356 |
A |2020-04-09T04:50:40.406000|Org1 |wifi |60 |662467 |
A |2020-04-10T00:00:50.636000|Org1 |remote|0 |72 |
A |2020-04-10T04:20:28.831000|Org1 |remote|6 |497 |
A |2020-04-10T04:31:35.336000|Org1 |remote|0 |22 |
B |2020-04-08T21:56:58.502000|Org2 |remote|0 |0 |
B |2020-04-08T22:01:19.534000|Org2 |wifi |0 |0 |
B |2020-04-08T22:10:15.891000|Org2 |wifi |60 |187891 |
B |2020-04-08T22:16:41.499000|Org2 |wifi |1620 |207674 |
B |2020-04-09T01:55:02.772000|Org2 |wifi |360 |426232 |
B |2020-04-09T02:03:32.735000|Org2 |wifi |60 |374827 |
B |2020-04-09T02:06:16.509000|Org2 |wifi |60 |386518 |
B |2020-04-09T02:13:33.497000|Org2 |remote|60 |373609 |
B |2020-04-09T02:17:19.176000|Org2 |wifi |133 |400417 |
+--------+--------------------------+-----+------+--------+-------+
only showing top 20 rows

In [12]:
w=Window().partitionBy("deviceId").orderBy(F.col("time-started").cast("long")).rangeBetween(Window.currentRow,24*60*60)
df2=df.withColumn("time-started", F.to_timestamp("time-started", "yyyy-MM-dd'T'HH:mm:ss"))\
      .withColumn("time-started-2", F.col("time-started"))\
      .withColumn("duration", F.sum("duration").over(w))\
      .withColumn("packet", F.sum("packet").over(w))


from pyspark.sql import functions as F
df2.groupBy("deviceId",F.window("time-started","1440 minutes"))\
  .agg(F.first("duration").alias("duration"),\
      F.first("packet").alias("packet")).show()

+--------+--------------------+--------+-------+
deviceId| window|duration| packet|
+--------+--------------------+--------+-------+
 B|[2020-04-08 00:00...| 2353|2357168|
 B|[2020-04-09 00:00...| 673|1961603|
 B|[2020-04-10 00:00...| 1| 497|
 A|[2020-04-08 00:00...| 23909|8213496|
 A|[2020-04-09 00:00...| 14950|4795979|
 A|[2020-04-10 00:00...| 6| 591|
+--------+--------------------+--------+-------+

In [13]:
+--------+-------------------+-----+------+--------+-------+
|deviceId|       time-started|OrgId|  type|duration| packet|
+--------+-------------------+-----+------+--------+-------+
|       A|2020-04-08 01:53:54| Org1|  wifi|   23909|8213496|
|       A|2020-04-09 04:27:33| Org1|remote|     186|1223887|
|       A|2020-04-10 04:31:35| Org1|remote|       0|     22|
|       B|2020-04-08 21:56:58| Org2|remote|    2353|2357168|
|       B|2020-04-10 23:10:15| Org2|remote|       1|    497|
+--------+-------------------+-----+------+--------+-------+

In [14]:
list=[['A','2020-04-08T01:53:54.932000','Org1','wifi',60,372717],
      ['A','2020-04-09T04:50:40.406000','Org1','wifi',60,662467],
                             ['A','2020-04-10T00:00:50.636000','Org1','remote',0,72],
                              ['A','2020-04-10T04:20:28.831000','Org1','remote',6,497],
                               ['A','2020-04-10T04:31:35.336000','Org1','remote',0,22],
                      ['A','2020-04-08T02:40:38.661000','Org1','wifi',194,819040],
                       ['A','2020-04-08T21:45:10.207000','Org1','wifi',8885,3449150],
                        ['A','2020-04-09T00:15:28.838000','Org1','wifi',14770,3572589],
                         ['A','2020-04-09T04:27:33.424000','Org1','remote',0,0],
                          ['A','2020-04-09T04:29:25.189000','Org1','wifi',60,7495],
                           ['A','2020-04-09T04:44:21.397000','Org1','remote',60,553356],
                               ['B','2020-04-09T01:55:02.772000','Org2','wifi',360,426232],
                                ['B','2020-04-08T21:56:58.502000','Org2','remote',0,0],
                                 ['B','2020-04-08T22:01:19.534000','Org2','wifi',0,0],
                                  ['B','2020-04-08T22:10:15.891000','Org2','wifi',60,187891],
                                   ['B','2020-04-08T22:16:41.499000','Org2','wifi',1620,207674],
                                     ['B','2020-04-09T02:03:32.735000','Org2','wifi',60,374827],
                                      ['B','2020-04-09T02:06:16.509000','Org2','wifi',60,386518],
                                       ['B','2020-04-09T02:13:33.497000','Org2','remote',60,373609],
                                        ['B','2020-04-09T02:17:19.176000','Org2','wifi',133,400417],
                                         ['B','2020-04-10T23:10:15.654000','Org2','remote',0,212],
                                          ['B','2020-04-10T23:10:41.749000','Org2','remote',1,285]]

df=spark.createDataFrame(list,["deviceId","time-started","OrgId","type","duration","packet"])
df.show()

+--------+--------------------+-----+------+--------+-------+
deviceId| time-started|OrgId| type|duration| packet|
+--------+--------------------+-----+------+--------+-------+
 A|2020-04-08T01:53:...| Org1| wifi| 60| 372717|
 A|2020-04-09T04:50:...| Org1| wifi| 60| 662467|
 A|2020-04-10T00:00:...| Org1|remote| 0| 72|
 A|2020-04-10T04:20:...| Org1|remote| 6| 497|
 A|2020-04-10T04:31:...| Org1|remote| 0| 22|
 A|2020-04-08T02:40:...| Org1| wifi| 194| 819040|
 A|2020-04-08T21:45:...| Org1| wifi| 8885|3449150|
 A|2020-04-09T00:15:...| Org1| wifi| 14770|3572589|
 A|2020-04-09T04:27:...| Org1|remote| 0| 0|
 A|2020-04-09T04:29:...| Org1| wifi| 60| 7495|
 A|2020-04-09T04:44:...| Org1|remote| 60| 553356|
 B|2020-04-09T01:55:...| Org2| wifi| 360| 426232|
 B|2020-04-08T21:56:...| Org2|remote| 0| 0|
 B|2020-04-08T22:01:...| Org2| wifi| 0| 0|
 B|2020-04-08T22:10:...| Org2| wifi| 60| 187891|
 B|2020-04-08T22:16:...| Org2| wifi| 1620| 207674|
 B|2020-04-09T02:03:...| Org2| wifi| 60| 374827|
 B|2020-04-09T02:06:...| Org2| wifi| 60| 386518|
 B|2020-04-09T02:13:...| Org2|remote| 60| 373609|
 B|2020-04-09T02:17:...| Org2| wifi| 133| 400417|
+--------+--------------------+-----+------+--------+-------+
only showing top 20 rows

In [15]:
import pandas as pd
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().partitionBy("deviceId").orderBy(F.col("time-started").cast("long")).rangeBetween(Window.currentRow,24*60*60)
df2=df.withColumn("time-started", F.to_timestamp("time-started", "yyyy-MM-dd'T'HH:mm:ss"))\
      .withColumn("time-started-2", F.col("time-started"))\
      .withColumn("duration", F.sum("duration").over(w))\
      .withColumn("packet", F.sum("packet").over(w))

@pandas_udf(df2.schema, PandasUDFType.GROUPED_MAP)
def grouped_map(df1):
   start=df1.loc[0, 'time-started']
   for i in range(1, len(df1)):
        if start + pd.Timedelta(days=1)>df1.loc[i,'time-started']:
             df1.loc[i,'time-started']=start
        else:
             start=df1.loc[i,'time-started']    
           

   return df1
df2.groupby('deviceId').apply(grouped_map)\
.filter(F.col("time-started-2")==F.col("time-started"))\
.drop("time-started-2")\
.orderBy("deviceId")\
.show()
  

+--------+-------------------+-----+------+--------+-------+
deviceId| time-started|OrgId| type|duration| packet|
+--------+-------------------+-----+------+--------+-------+
 A|2020-04-08 01:53:54| Org1| wifi| 23909|8213496|
 A|2020-04-09 04:27:33| Org1|remote| 186|1223887|
 A|2020-04-10 04:31:35| Org1|remote| 0| 22|
 B|2020-04-08 21:56:58| Org2|remote| 2353|2357168|
 B|2020-04-10 23:10:15| Org2|remote| 1| 497|
+--------+-------------------+-----+------+--------+-------+

In [16]:
w=Window().partitionBy("deviceId").orderBy(F.col("time-started").cast("long")).rangeBetween(Window.currentRow,24*60*60)
w1=Window().partitionBy("deviceId").orderBy(F.col("time-started"))
w2=Window().partitionBy("deviceId","sum").orderBy(F.col("time-started")).rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)

display(df.withColumn("time-started", F.to_timestamp("time-started", "yyyy-MM-dd'T'HH:mm:ss"))\
  .withColumn("packet1", F.sum("packet").over(w))\
  .withColumn("diff", F.col("time-started").cast("long")-F.lag("time-started").over(w1).cast("long"))\
  .withColumn("timelist", F.collect_list(F.col("time-started").cast("long")).over(w))\
  .withColumn("timelist2", F.collect_list(F.when(F.col("diff").isNotNull(),F.col("diff")).otherwise(F.lit(0))).over(w))\
  .select("deviceId","time-started","packet1","timelist","timelist2")\
    .withColumn("timelist2", F.expr("""transform(timelist2,(x,i)-> IF(i=0,0,x))"""))\
  .withColumn("agg", F.expr("""aggregate(timelist2, 0,(acc,t)->acc+int(t))"""))\
  .filter("deviceId='A'").orderBy("time-started"))

deviceId,time-started,packet1,timelist,timelist2,agg
A,2020-04-08T01:53:54.000+0000,8213496,"List(1586310834, 1586313638, 1586382310, 1586391328)","List(0, 2804, 68672, 9018)",80494
A,2020-04-08T02:40:38.000+0000,7840779,"List(1586313638, 1586382310, 1586391328)","List(0, 68672, 9018)",77690
A,2020-04-08T21:45:10.000+0000,8245057,"List(1586382310, 1586391328, 1586406453, 1586406565, 1586407461, 1586407840)","List(0, 9018, 15125, 112, 896, 379)",25530
A,2020-04-09T00:15:28.000+0000,4795979,"List(1586391328, 1586406453, 1586406565, 1586407461, 1586407840, 1586476850)","List(0, 15125, 112, 896, 379, 69010)",85522
A,2020-04-09T04:27:33.000+0000,1223887,"List(1586406453, 1586406565, 1586407461, 1586407840, 1586476850, 1586492428)","List(0, 112, 896, 379, 69010, 15578)",85975
A,2020-04-09T04:29:25.000+0000,1223887,"List(1586406565, 1586407461, 1586407840, 1586476850, 1586492428)","List(0, 896, 379, 69010, 15578)",85863
A,2020-04-09T04:44:21.000+0000,1216414,"List(1586407461, 1586407840, 1586476850, 1586492428, 1586493095)","List(0, 379, 69010, 15578, 667)",85634
A,2020-04-09T04:50:40.000+0000,663058,"List(1586407840, 1586476850, 1586492428, 1586493095)","List(0, 69010, 15578, 667)",85255
A,2020-04-10T00:00:50.000+0000,591,"List(1586476850, 1586492428, 1586493095)","List(0, 15578, 667)",16245
A,2020-04-10T04:20:28.000+0000,519,"List(1586492428, 1586493095)","List(0, 667)",667


In [17]:
x-> aggregate(timelist2, 0,(acc,t)->acc+t)

In [18]:
#`df.select('data_id',lpad(df['data'],length(df.data) +3 ,'p1/'))

list=[[1,'yo'],
      [2,'dpppp']]

df=spark.createDataFrame(list,['data_id','data'])


df.select("data_id",F.expr("""lpad(data,length(data)+3,'p1/')""")).show()

df.select("data_id",F.concat(F.lit('p1/'),F.col("data"))).show()

+-------+-----------------------------------+
data_id|lpad(data, (length(data) + 3), p1/)|
+-------+-----------------------------------+
 1| p1/yo|
 2| p1/dpppp|
+-------+-----------------------------------+

+-------+-----------------+
data_id|concat(p1/, data)|
+-------+-----------------+
 1| p1/yo|
 2| p1/dpppp|
+-------+-----------------+

In [19]:
from pyspark.sql.functions import window
df1=df.withColumn("time-started", F.to_timestamp("time-started", "yyyy-MM-dd'T'HH:mm:ss"))
df1.groupBy("deviceId",window(F.col("time-started"),"1 DAY")).agg(F.sum("packet")).show(truncate=False)

+--------+------------------------------------------+-----------+
deviceId|window |sum(packet)|
+--------+------------------------------------------+-----------+
B |[2020-04-08 00:00:00, 2020-04-09 00:00:00]|395565 |
A |[2020-04-09 00:00:00, 2020-04-10 00:00:00]|4795907 |
A |[2020-04-10 00:00:00, 2020-04-11 00:00:00]|591 |
B |[2020-04-09 00:00:00, 2020-04-10 00:00:00]|1961603 |
B |[2020-04-10 00:00:00, 2020-04-11 00:00:00]|497 |
A |[2020-04-08 00:00:00, 2020-04-09 00:00:00]|4640907 |
+--------+------------------------------------------+-----------+

In [20]:
LIST=[['key1',None,None,'v1'],
      [None,'key2',None,'v2'],
      [None,None,'key3','v3'],
      [None,'key4',None,'v4']]

df=spark.createDataFrame(LIST,['A','B','C','value'])

df.show()

+----+----+----+-----+
 A| B| C|value|
+----+----+----+-----+
key1|null|null| v1|
null|key2|null| v2|
null|null|key3| v3|
null|key4|null| v4|
+----+----+----+-----+

In [21]:
import pyspark.sql.functions as F

cols = [i for i in df.columns if i!='value'] #['A','B','C']

output = df.select(F.greatest(*[F.when(F.col(i).isNotNull(),i)
                             for i in cols]).alias("key_type")
               ,F.greatest(*[F.col(i) for i in cols]).alias("key_Value"),"value")

In [22]:
import pyspark.sql.functions as F

cols = [i for i in df.columns if i!='value'] #['A','B','C']

output = df.withColumn("key",F.greatest(*[F.when(F.col(i).isNotNull(),F.array(F.lit(i),F.col(i))) for i in cols]).alias("key")\
                      .select((F.col("key")[0])
               
output.show()

+---------+
 key|
+---------+
[A, key1]|
[B, key2]|
[C, key3]|
[B, key4]|
+---------+

In [23]:
df.withColumn("key",F.coalesce(*[(F.when(F.col(x).isNotNull(),F.array(F.lit(x),F.col(x)))).alias(x)\
                                 for x in df.columns if x!='value']))\
  .select((F.col("key")[0]).alias("key_type"),(F.col("key")[1]).alias("key_value"),F.col("value")).show()


+--------+---------+-----+
key_type|key_value|value|
+--------+---------+-----+
 A| key1| v1|
 B| key2| v2|
 C| key3| v3|
 B| key4| v4|
+--------+---------+-----+

In [25]:
df.withColumn("key",F.coalesce(*[(F.when(F.col(x).isNotNull(),F.array(F.lit(x),F.col(x))).otherwise(None)).alias(x)\
                                 for x in df.columns if x!='value']))\
  .select((F.col("key")[0]).alias("key_type"),(F.col("key")[1]).alias("key_value"),F.col("value")).show()

+--------+---------+-----+
key_type|key_value|value|
+--------+---------+-----+
 A| key1| v1|
 B| key2| v2|
 C| key3| v3|
 B| key4| v4|
+--------+---------+-----+

In [26]:
  select(F.coalesce(*["A","B","C"]).alias("key_value")).show()

In [27]:
list=[[  1,40,50],
      [  1,30,70],
      [  2,10,91],
      [  2,90,21],
      [ 3,20,42],
      [ 3,10,4],
      [  4,2,23],
      [ 4,5,12]]

df=spark.createDataFrame(list,['id','value1','value2'])
df.show()

+---+------+------+
 id|value1|value2|
+---+------+------+
 1| 40| 50|
 1| 30| 70|
 2| 10| 91|
 2| 90| 21|
 3| 20| 42|
 3| 10| 4|
 4| 2| 23|
 4| 5| 12|
+---+------+------+

In [29]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

w = Window().partitionBy()
df.groupBy("id").agg(F.sum("value1").alias("grouped_total_1"),F.sum("value2").alias("grouped_total_2"))\
          .withColumn("anti_grouped_total_1",F.sum("grouped_total_1").over(w)-F.col("grouped_total_1"))\
          .withColumn("anti_grouped_total_2",F.sum("grouped_total_2").over(w)-F.col("grouped_total_2")).orderBy("id").show()

+---+---------------+---------------+--------------------+--------------------+
 id|grouped_total_1|grouped_total_2|anti_grouped_total_1|anti_grouped_total_2|
+---+---------------+---------------+--------------------+--------------------+
 1| 70| 120| 137| 193|
 2| 100| 112| 107| 201|
 3| 30| 46| 177| 267|
 4| 7| 35| 200| 278|
+---+---------------+---------------+--------------------+--------------------+

In [30]:
list=[[1,'K'],
      [1,'A'],
       [1,'S'],
       [2,'M'],
       [2,'D'],
       [2,'S'],
       [3,'S'],
       [3,'S'],
       [4,'M'],
       [5,'K'],
       [1,'S'],
       [6,'S']]

df=spark.createDataFrame(list,['account_no','types'])

df.show()

+----------+-----+
account_no|types|
+----------+-----+
 1| K|
 1| A|
 1| S|
 2| M|
 2| D|
 2| S|
 3| S|
 3| S|
 4| M|
 5| K|
 1| S|
 6| S|
+----------+-----+

In [31]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().partitionBy("account_no")
w1=Window().partitionBy("account_no").orderBy("types")

df.withColumn("sum_S", F.sum(F.when(F.col("types")=='S', F.lit(1)).otherwise(F.lit(0))).over(w))\
  .withColumn("total", F.max(F.row_number().over(w1)).over(w))\
  .filter('total=sum_S').drop("total","Sum_S").show()

+----------+-----+
account_no|types|
+----------+-----+
 6| S|
 3| S|
 3| S|
+----------+-----+

In [32]:
w=Window().partitionBy("account_no")
w1=Window().partitionBy("account_no").orderBy("types")

df.withColumn("check_S", F.when(F.col("types")=='S', F.lit(1)).otherwise(F.lit(0)))\
  .withColumn("Sum_S", F.sum("check_S").over(w))\
  .withColumn("total", F.max(F.row_number().over(w1)).over(w))\
  .filter('total=Sum_S').drop("check_S","Sum_S").show()